<a href="https://colab.research.google.com/github/mralamdari/CV-Yolo/blob/main/YoloV3_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

# Yolo with COCO Dataset

## 1.Clone and build Darknet

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have gpu and opencv enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
#Build(make) Darknet
!make

## 2. Yolov3 Weights

In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights

In [ ]:
def imShow(path):
  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

## 3.Predict
    !./darknet detect <path to config> <path to weights> <path to image>

    imShow('predictions.jpg')



In [ ]:
!wget https://images.unsplash.com/photo-1583430999116-511887f6ee69?ixlib=rb-1.2.1&dl=donny-jiang-OWrC7nDbeM0-unsplash.jpg&q=80&fm=jpg&crop=entropy&cs=tinysrgb
!wget https://images.unsplash.com/photo-1483526682683-7560afbb5299?ixlib=rb-1.2.1&dl=elias-ehmann-eP93fjCLv4g-unsplash.jpg&q=80&fm=jpg&crop=entropy&cs=tinysrgb
!wget https://images.unsplash.com/photo-1520011554120-17c4c63fc213?ixlib=rb-1.2.1&dl=joseph-cooper-lEwc9W5eLH0-unsplash.jpg&q=80&fm=jpg&crop=entropy&cs=tinysrgb

In [ ]:
!./darknet detect cfg/yolov3.cfg yolov3.weights data/person.jpg

In [ ]:
imShow('predictions.jpg')

In [ ]:
# LOCAL MACHINE DOWNLOAD
download('predictions.jpg')

# GOOGLE DRIVE DOWNLOAD
!cp predictions.jpg /content/drive/MyDrive/Classroom/detection1.jpg

# Yolo with Custom Dataset

In [2]:
!git clone https://github.com/mralamdari/OIDv4_ToolKit.git

Cloning into 'OIDv4_ToolKit'...
remote: Enumerating objects: 452, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 452 (delta 2), reused 0 (delta 0), pack-reused 444
Receiving objects: 100% (452/452), 34.09 MiB | 35.02 MiB/s, done.
Resolving deltas: 100% (159/159), done.


In [ ]:
%cd OIDv4_ToolKit
!pip3 install -r requirements.txt

In [4]:
!python3 main.py downloader --classes Human_eye Human_nose Human_hair Human_mouth Human_ear Human_beard Human_face --type_csv train --limit 1000
# !python3 main.py downloader --classes Human_eye Human_nose Human_hair Human_mouth Human_ear Human_beard Human_face --type_csv train --limit 1000 --multiclasses 1
# !python3 main.py downloader --classes Human_eye Human_nose Human_hair Human_mouth Human_ear Human_beard Human_face --type_csv train --limit 5000 --multiclasses 1


		   ___   _____  ______            _    _    
		 .'   `.|_   _||_   _ `.         | |  | |   
		/  .-.  \ | |    | | `. \ _   __ | |__| |_  
		| |   | | | |    | |  | |[ \ [  ]|____   _| 
		\  `-'  /_| |_  _| |_.' / \ \/ /     _| |_  
		 `.___.'|_____||______.'   \__/     |_____|
	

             _____                    _                 _             
            (____ \                  | |               | |            
             _   \ \ ___  _ _ _ ____ | | ___   ____  _ | | ____  ____ 
            | |   | / _ \| | | |  _ \| |/ _ \ / _  |/ || |/ _  )/ ___)
            | |__/ / |_| | | | | | | | | |_| ( ( | ( (_| ( (/ /| |    
            |_____/ \___/ \____|_| |_|_|\___/ \_||_|\____|\____)_|    
                                                          
        
    [INFO] | Downloading Human eye.
   [ERROR] | Missing the class-descriptions-boxable.csv file.
[DOWNLOAD] | Do you want to download the missing file? [Y/n] Y
...145%, 0 MB, 32623 KB/s, 0 seconds passed
[DOWNLOAD] | Fil

In [5]:
!python3 convert_annotations.py

Currently in subdirectory: train
Converting annotations for class: Human ear
100% 1000/1000 [00:34<00:00, 29.11it/s]
Converting annotations for class: Human mouth
100% 1000/1000 [00:31<00:00, 31.69it/s]
Converting annotations for class: Human eye
100% 1000/1000 [00:54<00:00, 18.22it/s]
Converting annotations for class: Human beard
100% 1000/1000 [00:18<00:00, 54.51it/s]
Converting annotations for class: Human face
100% 1000/1000 [00:46<00:00, 21.31it/s]
Converting annotations for class: Human hair
100% 1000/1000 [01:10<00:00, 14.22it/s]
Converting annotations for class: Human nose
100% 1000/1000 [00:38<00:00, 26.22it/s]
